In [1]:
import os 
## Because the KERAS used is a old verion, KERAS 1
os.environ['TF_USE_LEGACY_KERAS'] = '1'

## Allows memory allocation on the GPU to be done asynchronously, 
## potentially improving performance and efficiency in certain scenarios.
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

## Specify which GPUs are available to a program.
## The list of GPUs is usually represented by indices starting at 0.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # disable cuda

# Install VideoImageTools

To install VideoImageTools go to next link https://github.com/trucomanx/VideoImageTools

# Install OpenPifPafTools

To install OpenPifPafTools go to next link https://github.com/trucomanx/OpenPifPafTools


In [2]:
input_default_json_conf_file='testing_over_video.json';

# Bibliotecas externas

In [3]:
import platform, sys, os

import json
import VideoImageTools as vit
import OpenPifPafTools.OpenPifPafAnnotations as opp
import OpenPifPafTools.OpenPifPafGetData as oppgd

import cv2
import openpifpaf
from PIL import Image

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-08-29 22:12:51.761595: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 22:12:52.301109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0


2024-08-29 22:12:52.713898: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-29 22:12:52.713914: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: fernando-B560M-DS3H-V2
2024-08-29 22:12:52.713918: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: fernando-B560M-DS3H-V2
2024-08-29 22:12:52.713946: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-08-29 22:12:52.713959: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.183.1
2024-08-29 22:12:52.713963: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 535.183.1


# Biblioteca local

In [5]:
sys.path.append('../library');

In [6]:
import SystemEmotion4Lib.Classifier as mylib

# Variables

In [7]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

In [8]:
## Model of network
checkpoint = DATA["checkpoint"];
model_type_body = DATA["model_type_body"]; ## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type_face = DATA["model_type_face"]; ## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type_skel = DATA["model_type_skel"];
model_type_fusion = DATA["model_type_fusion"];
enable_minus = DATA["enable_minus"];

vin_path = DATA['input_mp4_file'];

output_base_dir = DATA['output_base_dir'];

sub_dir='sub_dir1';

body_factor=1.0;
face_factor=0.85;


# If command line

In [9]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--model-type-check':
        checkpoint=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-body':
        model_type_body=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-face':
        model_type_face=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-skel':
        model_type_skel=int(sys.argv[n+1]);
    elif sys.argv[n]=='--model-type-fusion':
        model_type_fusion=int(sys.argv[n+1]);
    elif sys.argv[n]=='--enable-minus':
        enable_minus=sys.argv[n+1].lower()=='true';
    elif sys.argv[n]=='--input-file':
        vin_path=sys.argv[n+1];
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--sub-dir':
        sub_dir=sys.argv[n+1];


print('')
print('       checkpoint:',checkpoint);
print('  model_type_body:',model_type_body);
print('  model_type_face:',model_type_face);
print('  model_type_skel:',model_type_skel);
print('model_type_fusion:',model_type_fusion);
print('     enable_minus:',enable_minus);
print('         vin_path:',vin_path);
print('  output_base_dir:',output_base_dir);
print('          sub_dir:',sub_dir);
print('      body_factor:',body_factor);
print('      face_factor:',face_factor);
print('')


       checkpoint: shufflenetv2k16
  model_type_body: efficientnet_b3
  model_type_face: efficientnet_b3
  model_type_skel: 20
model_type_fusion: 11
     enable_minus: False
         vin_path: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/DATASET/TESE/PATIENT-RECOGNITION/PATIENT-VIDEOS/dataset-toy/drhouse_mini_cut.mp4
  output_base_dir: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1
          sub_dir: sub_dir1
      body_factor: 1.0
      face_factor: 0.85



In [10]:
vout_dir_path=os.path.join(output_base_dir,sub_dir,'test_over_video');
print('vout_dir_path:',vout_dir_path);

vout_dir_path: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/sub_dir1/test_over_video


In [11]:

vout_path=os.path.join(vout_dir_path,os.path.basename(vin_path));
print('vout_path:',vout_path);

vout_path: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/sub_dir1/test_over_video/drhouse_mini_cut.mp4


# Creating output directory

In [12]:
os.makedirs(vout_dir_path,exist_ok=True); 
print('Created output_dir:',vout_dir_path);

Created output_dir: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/sub_dir1/test_over_video


# Auxiliar functions

In [13]:
import numpy as np
import tensorflow as tf
import torch

def extra_func(cap):
    # Libera a memória da GPU no PyTorch
    torch.cuda.empty_cache()
    
    # Limpa a sessão atual e libera a memória da GPU no TensorFlow
    tf.keras.backend.clear_session()
    print('clean')

def my_func(Clf,frame):
    #categories=['angry','disgusted','fearful','happy','neutral','sad','surprised'];
    categories=['negative','neutral','pain','positive'];
    
    img_tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB);
    pil_im = Image.fromarray(img_tmp);
    
    res, res_face, res_body, res_skel, face_bbox, body_bbox = Clf.predict_all_pil(pil_im);

    if res is None:
        return frame;
    
    res = np.argmax(res);
    texto=categories[res];
    (xi,yi,xo,yo) = body_bbox;
    color=(0,255,0);
    thickness=2;
    
    frame = cv2.putText(frame,
                        texto,
                        org = (int(xi), int((yi+yo)/2)),
                        fontFace = cv2.FONT_HERSHEY_DUPLEX,
                        fontScale = 2.0,
                        color = (255, 0, 0),
                        thickness = thickness
                        );

    cv2.rectangle(frame,(xi,yi),(xo,yo),color,thickness);
    return frame;




In [ ]:
Clf=mylib.Emotion4Classifier(   checkpoint=checkpoint,
                                model_type_face=model_type_face,
                                model_type_body=model_type_body,
                                model_type_skel=model_type_skel,
                                model_type_skel_enable_minus=enable_minus,
                                model_type_fusion=model_type_fusion,
                                body_factor=body_factor,
                                face_factor=face_factor);

vit.apply_func_predictor_over_video_outmp4(my_func,Clf,vin_path,vout_path,extra_func_counter=1024,extra_func=extra_func)

'''
## Crio pasata com imagens
vout_dir_tmp=os.path.join(vout_dir_path,os.path.splitext(os.path.basename(vin_path))[0]+'_'+model_type);
image_files, fps = vit.apply_func_predictor_over_video_to_frames(my_func,(Clf,predictor),vin_path,vout_dir_tmp, show=False,FORMATO = "frame_{:05d}.png");

## Save frames to video
vit.images_to_video(image_files, fps, vout_path)
'''

print('working end')


Transfer learning architecture efficientnet_b3
        url: https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1
target_size: (300, 300)
Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/FaceEmotion4Lib/models/model_efficientnet_b3.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/FaceEmotion4Lib/models/model_efficientnet_b3.h5

Transfer learning architecture efficientnet_b3
        url: https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1
target_size: (300, 300)
Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/BodyEmotion4Lib/models/model_efficientnet_b3.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/BodyEmotion4Lib/models/model_efficientnet_b3.h5

Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/SkeletonEmotion4Lib/models/model_onlycls_ncod20.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/SkeletonEmotion

Working:  11%|████████████████▌                                                                                                                                        [ time left: 12:23 ]

No person was found, will be returned All None.
No person was found, will be returned All None.


Working:  11%|████████████████▋                                                                                                                                        [ time left: 10:14 ]

No person was found, will be returned All None.
No person was found, will be returned All None.


...

Working:  15%|███████████████████████▏                                                                                                                                 [ time left: 07:48 ]

No person was found, will be returned All None.
No person was found, will be returned All None.


Working:  15%|███████████████████████▏                                                                                                                                 [ time left: 07:48 ]

No person was found, will be returned All None.


Working:  16%|███████████████████████▊                                                                                                                                 [ time left: 15:43 ]